In [3]:
!pip install datasets evaluate transformers[sentencepiece]

  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

In [8]:
raw_inputs = [
    "I'm grateful for all the happiness in my life.",
    "This is really a bad product.",
    "I thought it was good, but it's actuallly not!"
]

In [9]:
classifier = pipeline("sentiment-analysis")
classifier(
    raw_inputs
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998650550842285},
 {'label': 'NEGATIVE', 'score': 0.9998207688331604},
 {'label': 'NEGATIVE', 'score': 0.9983192086219788}]

In [12]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

inputs = tokenizer(raw_inputs, truncation=True, padding=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 1045, 1005, 1049, 8794, 2005, 2035, 1996, 8404, 1999, 2026, 2166,
         1012,  102,    0,    0],
        [ 101, 2023, 2003, 2428, 1037, 2919, 4031, 1012,  102,    0,    0,    0,
            0,    0,    0,    0],
        [ 101, 1045, 2245, 2009, 2001, 2204, 1010, 2021, 2009, 1005, 1055, 5025,
         9215, 2025,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [13]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [14]:
outputs.logits.shape

torch.Size([3, 2])

In [17]:
print(outputs.logits)

tensor([[-4.2732,  4.6375],
        [ 4.7966, -3.8300],
        [ 3.5304, -2.8564]], grad_fn=<AddmmBackward0>)


In [21]:
# need to softmax the logits
import torch

preds = torch.nn.functional.softmax(outputs.logits, dim=-1) # -1 -> softmax across the last dimension(columns)(multi-class)
print(preds)

tensor([[1.3493e-04, 9.9987e-01],
        [9.9982e-01, 1.7924e-04],
        [9.9832e-01, 1.6808e-03]], grad_fn=<SoftmaxBackward0>)


In [23]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}